In [ ]:
#for relative open masslib
import sys
sys.path.append('..')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nhsmasslib.mass import MassSpectrum
from nhsmasslib.mass import VanKrevelen
from nhsmasslib.mass import Tmds
from nhsmasslib.mass import Reaction
from nhsmasslib.brutto_generator import brutto_gen

#### Discovery difference of soruce with product

In this example source was modifed by hydroquinone. So in product we expect to find source peak + quinone mass.

#### Load source

In [ ]:
sourse = MassSpectrum().load('data/sourse.csv', mapper={'m/z':'mass','I':'intensity'}, sep=',', take_columns=['mass','intensity'])
gdf = brutto_gen(elems={'C':(1,40),'H':(0,80), 'O':(0,40),'N':(0,2)})
sourse = sourse.assign(generated_bruttos_table=gdf, rel_error=0.5)
vk = VanKrevelen(sourse.table).draw_scatter(nitrogen=True)

#### Load product

In [ ]:
product = MassSpectrum().load('data/product.csv', mapper={'m/z':'mass','I':'intensity'}, sep=',', take_columns=['mass','intensity'])
gdf = brutto_gen(elems={'C':(1,40),'H':(0,80), 'O':(0,40),'N':(0,2)})
product = product.assign(generated_bruttos_table=gdf, rel_error=0.5)
vk = VanKrevelen(product.table).draw_scatter(nitrogen=True)

#### We can look difference spectrum

In [ ]:
vk = VanKrevelen(product - sourse).draw_scatter(nitrogen=True)

#### Now calculate total mass differnce statistic

We use restrict brutto gen because we expect to find six carbon atom difference. But you can study another atoms ranges

In [ ]:
tmds_spec = Tmds().calc(sourse, product, p=0.2) #by varifiy p-value we can choose how much mass-diff we will take
tmds_spec = tmds_spec.assign(gdf={'C':(6,7),'H':(0,13), 'O':(0,5)})
tmds_spec = tmds_spec.calculate_mass()
tmds_spec.draw()

In [ ]:
tmds_spec.table.sort_values(by='probability', ascending=False)

Now we can see that most abudant mass-difference is C4H4O2, that contain in quinone fragment. We can propose that modification by quinone is occure.

#### Find source and product peaks that was modifed

Now we look which peaks in sourse was modifed by C4H4O2 fragment, and find them in product.

In [ ]:
gdf = brutto_gen(elems={'C':(4,5),'H':(4,5), 'O':(2,3)})

In [ ]:
gdf

In [ ]:
reaction = Reaction(sourse=sourse, product=product)

In [ ]:
reaction = reaction.find_modification(gdf)

In [ ]:
reaction.draw_modification()